In [1]:
from music21 import converter, instrument, note, chord
import glob
import pickle
import numpy as np
from keras.utils import np_utils
import tensorflow as tf

/Users/chi/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# import tensorflow module

In [3]:
def get_notes(file_path):
    notes = []
    for file in glob.glob(file_path):
        midi = converter.parse(file)
        notes_to_parse = None
        parts = instrument.partitionByInstrument(midi)
        if parts: # file has instrument parts
            notes_to_parse = parts.parts[0].recurse()
        else: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes
        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))
        
        with open('data/notes', 'wb') as filepath:
            pickle.dump(notes, filepath)
    return notes

In [4]:
# greate notes from midi files
file_path = "../tutorial/Classical-Piano-Composer/midi_songs/*.mid"
notes = get_notes(file_path)

# get amount of  picthnames
pitchnames = sorted(set(notes))
n_vocab = len(pitchnames)

In [5]:
def create_data (notes, pitchnames, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100
    
     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

In [6]:
# get data
network_input, network_output = create_data(notes, pitchnames, n_vocab)

In [7]:
network_input.shape # [None, n_steps, n_inputs]  n_steps = 100, n_inputs = 1, n_classes = n_vocab

(57077, 100, 1)

In [8]:
lr = 0.001
training_iters = 100
batch_size = 128 # 1 エポックで　57077 % 128 回投入

n_inputs = 1   # MNIST data input (img shape: 28*28), specify columns
n_steps = 100    # time steps, specify rows
n_hidden_units = 512   # neurons in hidden layer
n_classes = n_vocab      # MNIST classes (0-9 digits)

In [9]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.float32, [None, n_classes])

# Define weights
weights = {
    # (1, 512)
    'in': tf.Variable(tf.random_normal([n_inputs, n_hidden_units])),
    # (512, n_vocab)
    'out': tf.Variable(tf.random_normal([n_hidden_units, n_classes]))
}
biases = {
    # (512, )
    'in': tf.Variable(tf.constant(0.1, shape=[n_hidden_units, ])),
    # (n_vocab, )
    'out': tf.Variable(tf.constant(0.1, shape=[n_classes, ]))
}

In [ ]:
def RNN(X, weights, biases):
    # hidden layer for input to cell
    # transpose the inputs shape from
    # X ==> (128 batch * 28 steps, 28 inputs)
    X = tf.reshape(X, [-1, n_inputs])

    # into hidden
    # X_in = (128 batch * 28 steps, 128 hidden)
    X_in = tf.matmul(X, weights['in']) + biases['in']
    # X_in ==> (128 batch, 28 steps, 128 hidden)
    X_in = tf.reshape(X_in, [-1, n_steps, n_hidden_units])

    # basic LSTM Cell.
    if int((tf.__version__).split('.')[1]) < 12 and int((tf.__version__).split('.')[0]) < 1:
        cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden_units, forget_bias=1.0, state_is_tuple=True)
    else:
        cell = tf.contrib.rnn.BasicLSTMCell(n_hidden_units)
    # lstm cell is divided into two parts (c_state, h_state)
    init_state = cell.zero_state(batch_size, dtype=tf.float32)

    outputs, final_state = tf.nn.dynamic_rnn(cell, X_in, initial_state=init_state, time_major=False)

    # # or
    # unpack to list [(batch, outputs)..] * steps
    if int((tf.__version__).split('.')[1]) < 12 and int((tf.__version__).split('.')[0]) < 1:
        outputs = tf.unpack(tf.transpose(outputs, [1, 0, 2]))    # states is the last outputs
    else:
        outputs = tf.unstack(tf.transpose(outputs, [1,0,2]))
    results = tf.matmul(outputs[-1], weights['out']) + biases['out']    # shape = (128, 10)

    return results